In [1]:
! pip install selenium
! pip install webcolors
! pip install --upgrade google-cloud-bigquery

! pip install validators

! apt-get update
! apt install chromium-chromedriver
! cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 954 kB 6.3 MB/s eta 0:00:01
  Using cached urllib3-1.26.7-py2.py3-none-any.whl (138 kB)
     |████████████████████████████████| 356 kB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 8.8 MB/s  eta 0:00:01
ERROR: conda 4.10.3 requires ruamel_yaml_conda>=0.11.14, which is not installed.
ERROR: requests 2.24.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.7 which is incompatible.
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.9
    Uninstalling urllib3-1.25.9:
      Successfully uninstalled urllib3-1.25.9
     |████████████████████████████████| 203 kB 9.2 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 9.3 MB/s  eta 0:00:01
     |████████████████████████████████| 95 kB 1.5 MB/s eta 0:00:011
     |████████████████████████████████| 45 kB 7.8 MB/s  eta 0:00:01
     |████████████████████████████████| 127 kB 6.2 MB/s eta 0:00:01
     |███████████████

  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.7
    Uninstalling urllib3-1.26.7:
      Successfully uninstalled urllib3-1.26.7
zsh:1: command not found: apt-get
The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

cp: /usr/lib/chromium-browser/chromedriver: No such file or directory


In [7]:
! pip install bigquery

     |████████████████████████████████| 171 kB 8.9 MB/s eta 0:00:01
     |████████████████████████████████| 11.4 MB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 19.1 MB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 8.2 MB/s  eta 0:00:01
     |████████████████████████████████| 142 kB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 8.0 MB/s  eta 0:00:01
     |████████████████████████████████| 514 kB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 206 kB 11.2 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 9.9 MB/s  eta 0:00:01
     |████████████████████████████████| 380 kB 5.5 MB/s eta 0:00:01


  Created wheel for bigquery: filename=bigquery-0.0.20-py3-none-any.whl size=9161 sha256=b03f1a816abc050c4c522dd7a90f37b61b7b5b194316986328b2c8baf3605087
  Stored in directory: /Users/doguma/Library/Caches/pip/wheels/e1/99/21/a4b265bd46044ed58504686a85c2e787ba02cb7dfa7fb443e2
  Created wheel for dbstream: filename=dbstream-0.1.10-py3-none-any.whl size=7502 sha256=579d9854f90a3e556145aa72ab51e5ffbf8289b61f0d6b2817029fae3fe67de7
  Stored in directory: /Users/doguma/Library/Caches/pip/wheels/5b/5a/39/c988bd861bbcff0deec6086dd1f413f339c88d18e80c7d767e
  Created wheel for googleauthentication: filename=googleauthentication-0.0.16-py3-none-any.whl size=4735 sha256=d6bbfb8a8df57115cc18fab29b5678602722840950a882ccdb0455e6f589016a
  Stored in directory: /Users/doguma/Library/Caches/pip/wheels/8a/c3/b1/37190b8ddc140af443c19428a896c65e47f0f53aa32e2875a4
  Created wheel for dacktool: filename=dacktool-0.0.7-py3-none-any.whl size=4802 sha256=39f403bf70765ebaac5416ed76c8c3727bed97f70b35edb6a5f0450a1

In [14]:
import os, time, sys, io
import cv2, keras, glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import Counter
from numpy import asarray
from io import BytesIO

from google.oauth2 import service_account
from google.cloud import bigquery

import requests
import PIL
import re
import urllib
import urllib.request
import webcolors
import validators
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from PIL import Image, ImageColor

from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
def start_chromedriver():
    sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    
    return driver

In [2]:
# Hex to RGB Conversion
# Source: https://gist.github.com/kb22/f17e59a79d4fcca02188c23cca932be5#file-rgb2hex-py

def hex2name(c):
    h_color = '#{:02x}{:02x}{:02x}'.format(int(c[0]), int(c[1]), int(c[2]))

    rms_lst = []
    for img_clr, img_hex in webcolors.CSS3_NAMES_TO_HEX.items():
        cur_clr = webcolors.hex_to_rgb(img_hex)
        rmse = np.sqrt(mean_squared_error(c, cur_clr))
        rms_lst.append(rmse)

    closest_color = rms_lst.index(min(rms_lst))

    nm = list(webcolors.CSS3_NAMES_TO_HEX.items())[closest_color][0]

    return nm

In [8]:
# connecting to google cloud platform

! gcloud iam service-accounts create dopoem
! gcloud projects add-iam-policy-binding dopoem --member="serviceAccount:dopoem@dopoem.iam.gserviceaccount.com" --role="roles/owner"
! gcloud auth login

In [9]:
! export GOOGLE_APPLICATION_CREDENTIALS="/home/user/Downloads"
! export GOOGLE_APPLICATION_CREDENTIALS="/home/user/Downloads/service-account-file.json"

In [18]:
# get json file from google cloud platform

credentials = "dopoem-f5aa6b994716.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials

In [19]:
# Google Cloud's client to pull data

client = bigquery.Client()

In [22]:
# Querying images and vision_api_data from "bigquery-public-data"

QUERY4 = ("""
SELECT * FROM (
  SELECT *
  FROM bigquery-public-data.the_met.images img
  LEFT JOIN (
    SELECT 
        *
    FROM bigquery-public-data.the_met.vision_api_data
  ) vis
  ON img.object_id = vis.object_id
)
LIMIT 700  # 500 for train, 200 for test
""")

In [23]:

met_images, met_id, type1, type2, type3, type4, color1, color2, color3, title, text = [], [], [], [], [], [], [], [], [], [], []

query_job4 = client.query(QUERY4)
rows4 = query_job4.result()


def url_to_image(link):
  response = requests.get(link)
  # get image links that only respond with status code of 200
  # ignore the 'not found' pages
  if response.status_code == 200:
    image_bytes = io.BytesIO(response.content)
    img = PIL.Image.open(image_bytes)
    return img
  else:
    return None


for i in rows4:

      image = url_to_image(i[3])
      if image != None:
        met_id.append(i[0])
        img_numpy = asarray(image)
        met_images.append(img_numpy)

        # driver = start_chromedriver()
        # driver.get('https://www.metmuseum.org/art/collection/search/' + str(i[0]))
        # html = driver.page_source

        # soup = BeautifulSoup(html, 'html.parser')
  
        # title.append(soup.find('span', 'artwork__title__inset').text)
        # text.append(soup.find('div', 'artwork__intro__desc').text)

        # driver.quit()

        # try and except due to missing values
        try:
          type1.append(i[12][0]['description'])
        except:
          type1.append(None)
        try:
          type2.append(i[12][1]['description'])
        except:
          type2.append(None)
        try:
          type3.append(i[12][2]['description'])
        except:
          type3.append(None)
        try:
          type4.append(i[12][3]['description'])
        except:
          type4.append(None)

        # I should use functions more next time..
        color1.append(hex2name((i[15]['dominantColors']['colors'][0]['color']['blue'],\
                i[15]['dominantColors']['colors'][0]['color']['green'],\
                i[15]['dominantColors']['colors'][0]['color']['red'])))

        color2.append(hex2name((i[15]['dominantColors']['colors'][1]['color']['blue'],\
                i[15]['dominantColors']['colors'][1]['color']['green'],\
                i[15]['dominantColors']['colors'][1]['color']['red'])))
        
        color3.append(hex2name((i[15]['dominantColors']['colors'][2]['color']['blue'],\
                i[15]['dominantColors']['colors'][2]['color']['green'],\
                i[15]['dominantColors']['colors'][2]['color']['red'])))

In [24]:
np.save('met_total_images', met_images)

/Users/doguma/anaconda3/lib/python3.8/site-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [36]:
met_collec_data = None
met_collec_data = pd.DataFrame()
met_collec_data['id'] = met_id
met_collec_data['type1'] = type1
met_collec_data['type2'] = type2
met_collec_data['type3'] = type3
met_collec_data['type4'] = type4
met_collec_data['color1'] = color1
met_collec_data['color2'] = color2
met_collec_data['color3'] = color3

In [ ]:
np.save('met_total_images', met_images)

In [63]:
met_data = None
met_data = pd.DataFrame()
met_data['id'] = met_id
met_data['type1'] = type1
met_data['type2'] = type2
met_data['type3'] = type3
met_data['color1'] = color1
met_data['color2'] = color2
met_data['color3'] = color3

In [86]:
def clean_list(sample_list):
    temp = []
    for val in sample_list:
        if val != None :
            temp.append(val)
        else :
            temp.append(0)
    return temp

In [87]:
test = clean_list(met_data['type3'])

In [99]:
import pandas as pd

met_total_data = pd.DataFrame()

met_total_data['id'] = clean_list(met_data['id'])
met_total_data['type1'] = clean_list(met_data['type1'])
met_total_data['type2'] = clean_list(met_data['type2'])
met_total_data['type3'] = clean_list(met_data['type3'])
met_total_data['color1'] = clean_list(met_data['color1'])
met_total_data['color2'] = clean_list(met_data['color2'])
met_total_data['color3'] = clean_list(met_data['color3'])

In [108]:
import pickle

In [109]:
filename = 'met_total_data'

outfile = open(filename, 'wb')

pickle.dump(met_total_data, outfile)
outfile.close()

In [ ]:
# def remove_space(string):
#     return re.sub(r'\n', '', string)

# met_collec_data['text'] = met_collec_data['text'].apply(remove_space)
# met_collec_data['title'] = met_collec_data['title'].apply(remove_space)